In [0]:
import numpy as np


def compat_conj_classes_of_subgroups(G,N):
    """Input: a gap group G and normal subgroup N
    Output: a list conjugacy classes of subgroups [S] such that N < S < G."""
    compats_list = []
    for conjclass in gap.List(gap.ConjugacyClassesSubgroups(G)):
        if bool(gap.IsSubgroup(conjclass[1],N)):
            compats_list.append(conjclass)
    return compats_list


def H_representations_up_to_normaliser(G,H):
    """Input: a gap group G and subgroup H
    Output: a list of representatives of orbits of representations of H under action by normaliser N_G(H)."""

    def index_of_value_in_lists(value, list_of_lists):
        """Input: an element 'value' we wish to locate, and a list of lists
        Output: a number i where i is the index of the list for which value first lies
        (We will need this to determine how action of H behaves on conj classes of H)"""
        for index, sublist in enumerate(list_of_lists,1):
            if value in sublist:
                return index
        return None

    char_table = gap.CharacterTable(H)
    conj_classes_H = gap.ConjugacyClasses(char_table)
    #We compute conjugacy classes using the character table above so that the order of the list of conj classes agrees with order of columns of char table.
    characters = gap.Irr(H)

    char_to_be_deleted = [] #this will contain indices of characters which are equivalent to another character and should be removed.

    for i in range(1,len(characters)):
        for j in range(i+1,len(characters)+1):
            for x in gap.List(gap.Normaliser(G,H)):
                equal = True
                for index, ele in enumerate(gap.List(conj_classes_H,gap.Representative),1):
                    index_of_ele = index
                    index_of_x_ele_xinverse = index_of_value_in_lists(x*ele*gap.Inverse(x),gap.List(conj_classes_H))
                    if characters[i][index_of_ele] != characters[j][index_of_x_ele_xinverse]:
                        equal = False
                        break
                if equal == True:
                    char_to_be_deleted.append(i)
                    break
            if i in char_to_be_deleted:
                #character at index i is already going to be deleted so no need to keep comparing it to other characters.
                break
    return [characters[i] for i in range(1,len(characters)+1) if i not in char_to_be_deleted]


def representatives_of_normaliser_acting_on_H(G,H):
    """Input: gap groups G and H
    Output: a list of representatives for orbits under action of N_G(H) on H
    (key Observation. Orbits_{N_G(H)}(H) = conj_classes(N_G(H)) intersection H)"""

    orbits = []

    normaliser = gap.Normaliser(G,H)
    conj_classes_normaliser = gap.ConjugacyClasses(normaliser)
    for conj_classes in gap.List(conj_classes_normaliser):
        if gap.Size(gap.Intersection(conj_classes,H)) > 0:
            orbits.append(conj_classes)
    return [gap.List(ele)[1] for ele in orbits]


def fixed_points_K_acting_G_H(G,H,K):
    """Input: a group G and subgroups H, K.
    Output: A list of representatives in G for (G/H)^K."""

    for S in gap.Subgroups(G):
        if gap.Size(S) == 1:
            ident = S #ident = identity subgroup to be used later.
            break

    fixed_pt_representatives = []

    representatives_of_G_H = [gap.Inverse(x) for x in gap.List(gap.RightCosets(G,H),gap.Representative)]
    #Note on above: we want left cosets, gap computes right cosets so use xH <-> Hx^{-1}

    #gH in (G/H)^K if and only if KgH = gH if and only if KgH = {e}gH as double cosets.
    for g in representatives_of_G_H:
        if gap.Size(gap.DoubleCoset(K,g,H)) == gap.Size(gap.DoubleCoset(ident,g,H)): #i.e. KgH = gH (since gH \subset KgH)
            fixed_pt_representatives.append(g)

    return fixed_pt_representatives


def pretty_print(T):
    """Input: a 2d array with data types which are able to be represented by a string. 
    Output: prints the array nicely, i.e. with columns aligned."""
    Tstr = [[str(ele) for ele in row] for row in T]
    lens = [max(map(len, col)) for col in zip(*Tstr)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in Tstr]
    print('\n'.join(table))


def reduced_global_table(G,N,print_it = True):
    """Input: a gap group G and normal subgroup H and boolean print_it
    Output: if print_it = False: a numpy 2d array - this is the reduced global table T(G,N)
    if print_it = True, no return value, but prints the table nicely (see function pretty_print)."""

    #First we create dictionaries which hold the representatives for characters and representatives for elements in each compatible subgroup.
    compat_ccl_of_subgroups = compat_conj_classes_of_subgroups(G,N)
    compats = [ccl[1] for ccl in compat_ccl_of_subgroups]
    num_of_compats = len(compats) #the global table will be a block 'num_of_compats' x 'num_of_compats' table.

    Representatives_of_characters_of_subgroups = {H: H_representations_up_to_normaliser(G,H) for H in compats}
    Representatives_of_elements_of_subgroups = {H : representatives_of_normaliser_acting_on_H(G,H) for H in compats}

    #For each k \in K, we calculate representatives g for (G/H)^K. For each g^{-1}kg we want to know which conjugacy class of N it is in.
    #If it is not in any conjugacy class attach "none" to it

    table = []

    for index, H in enumerate(compats):
        representative_char_table = Representatives_of_characters_of_subgroups[H] #retrieve table from dict above
        representative_char_table_transposed = list(zip(*representative_char_table))
        height_H = len(representative_char_table) #height of any block in the table of the form B_{H,-}
        conj_classes_of_H = gap.ConjugacyClasses(gap.CharacterTable(H))

        for jndex, K in enumerate(compats):
            representatives_of_elements = Representatives_of_elements_of_subgroups[K]
            length_K = len(representatives_of_elements) #length of any block in the table of the form B_{-,K}

            if gap.Size(K) > gap.Size(H): #if True the block B_{H,K} will be a zero block.
                table.append(np.zeros((height_H, length_K),dtype=int))

            elif True not in [bool(gap.IsSubset(sub,K)) for sub in gap.List(compat_ccl_of_subgroups[index])]: #K not subgroup of gHg^{-1} for any g \in G
                table.append(np.zeros((height_H, length_K),dtype=int))

            else:
                block_transpose = [] #this will be the block in the global table B_{H,K} (transposed)
                fixed_pts = fixed_points_K_acting_G_H(G,H,K) #by above if/elif this will always be non empty.

                for k in representatives_of_elements:
                    sum_of_columns_of_char = None
                    for g in fixed_pts:
                        for counter in range(1, int(gap.Size(conj_classes_of_H))+1): #gap indexing starts at 1
                            if gap.Inverse(g)*k*g in conj_classes_of_H[counter]:
                                if sum_of_columns_of_char == None:
                                    sum_of_columns_of_char = representative_char_table_transposed[counter-1] #this list starts with index 0...
                                else:
                                    sum_of_columns_of_char = [x + y for x, y in zip(sum_of_columns_of_char, representative_char_table_transposed[counter-1])]
                                break #g^{-1}kg can only be in one conjugacy class, so once its found no need to check the rest.
                    block_transpose.append(np.array(sum_of_columns_of_char))
                block = list(zip(*block_transpose))
                table.append(np.array(block))
    #Now table is a list of n^2 blocks (n = num_of_compats). We want a block n x n table. We start by creating the rows by hstacking blocks together.
    table_v2 = []
    for n in range(num_of_compats):
        table_v2.append(np.hstack([table[j] for j in range(n*num_of_compats,(n+1)*num_of_compats)]))

    #Table_v2 is a List of lists ("big rows"/horizontal blocks) of lists (the rows within blocks) of lists (elements in each row)
    #i.e table_v2 = [[[...]],[[...],[...]],...]. This is a little much. Let's combine all vertical blocks together with vstack.

    table_v3 = np.vstack([table_v2[i] for i in range(num_of_compats)])

    if print_it == True:
        pretty_print(table_v3)
    else:
        return table_v3